# Understanding the Artificial Bee Colony Algorithm

The Artificial Bee Colony (ABC) algorithm is an optimization technique inspired by the natural foraging behavior of honey bees, designed to tackle complex optimization challenges by leveraging the collective intelligence arising from individual bees' simple actions. Noted for its straightforwardness, adaptability, and resilience, the ABC algorithm finds applications across a broad spectrum of optimization tasks.

This algorithm models three bee roles: employed bees, onlooker bees, and scout bees. Employed bees focus on refining the nectar sources (solutions) they've discovered and communicate this information to onlooker bees. These onlooker bees evaluate nectar sources based on their quality or fitness, examining nearby areas for potentially superior solutions. Scout bees explore new regions at random to prevent the search from lingering in local optima, serving as the algorithm's method for exploration.

The ABC algorithm initiates with a set of initial solutions, each representing a nectar source. It seeks to enhance these solutions iteratively, mimicking the activities of the three bee types, with the aim of uncovering the optimal solution or the finest nectar source relative to the problem addressed. This entails a careful balance between exploration, to uncover new search areas, and exploitation, to benefit from identified optimal solutions.

A key advantage of the ABC algorithm is its proficiency in navigating problems characterized by multiple local optima, attributed to its dual focus on exploration and exploitation. Its successful deployment in various fields, such as engineering, scheduling, and machine learning, underlines its broad applicability and efficacy in resolving intricate optimization issues.

Customizing the ABC algorithm for particular problems involves adjusting parameters like the bee colony size, the threshold for activating scout bees, and the selection process for nectar sources by onlooker bees. With precise adjustments and a deep understanding of the specific problem, the ABC algorithm can yield effective optimization solutions, demonstrating its potential in addressing a wide range of complex challenges.

In [1]:
import numpy as np

# Problem parameters
W = 5  # Number of workers
T = 10  # Number of tasks
efficiency_matrix = np.random.uniform(1, 10, (W, T))  # Efficiency of each worker for each task

# ABC parameters
colony_size = 50
max_cycles = 100
limit = 50  # Scout bee limit

# Decision variable bounds
lb = 0
ub = 1

# Initialization
def initialize_population():
    population = []
    for _ in range(colony_size):
        solution = np.zeros((W, T))
        for t in range(T):
            worker = np.random.randint(0, W)
            solution[worker, t] = 1
        population.append(solution)
    return population

def calculate_fitness(solution):
    total_efficiency = 0
    for t in range(T):
        for w in range(W):
            if solution[w, t] == 1:
                total_efficiency += efficiency_matrix[w, t]
    return total_efficiency

def employed_bees_phase(population):
    for i, solution in enumerate(population):
        # Choose a solution index different from i
        k = np.random.randint(0, colony_size)
        while k == i:
            k = np.random.randint(0, colony_size)
        
        # Choose a random task to modify
        t = np.random.randint(0, T)
        
        # Generate a new solution by changing the worker assignment for task t
        new_solution = np.copy(solution)
        current_worker = np.argmax(solution[:, t])
        new_worker = np.random.randint(0, W)
        while new_worker == current_worker:
            new_worker = np.random.randint(0, W)
        
        new_solution[current_worker, t] = 0
        new_solution[new_worker, t] = 1
        
        # Greedy selection
        if calculate_fitness(new_solution) > calculate_fitness(solution):
            population[i] = new_solution

def onlooker_bees_phase(population):
    fitness = [calculate_fitness(sol) for sol in population]
    prob = [f / sum(fitness) for f in fitness]
    for _ in range(colony_size):
        i = np.random.choice(range(colony_size), p=prob)
        solution = population[i]
        
        # Employed bee phase-like operation
        k = np.random.randint(0, colony_size)
        while k == i:
            k = np.random.randint(0, colony_size)
        
        t = np.random.randint(0, T)
        
        new_solution = np.copy(solution)
        current_worker = np.argmax(solution[:, t])
        new_worker = np.random.randint(0, W)
        while new_worker == current_worker:
            new_worker = np.random.randint(0, W)
        
        new_solution[current_worker, t] = 0
        new_solution[new_worker, t] = 1
        
        if calculate_fitness(new_solution) > calculate_fitness(solution):
            population[i] = new_solution

def scout_bees_phase(population, trial_counter):
    for i, trial in enumerate(trial_counter):
        if trial > limit:
            new_solution = np.zeros((W, T))
            for t in range(T):
                worker = np.random.randint(0, W)
                new_solution[worker, t] = 1
            population[i] = new_solution
            trial_counter[i] = 0

# ABC Algorithm
def abc_algorithm():
    population = initialize_population()
    trial_counter = [0] * colony_size
    best_solution = None
    best_fitness = float('-inf')
    
    for _ in range(max_cycles):
        employed_bees_phase(population)
        onlooker_bees_phase(population)
        scout_bees_phase(population, trial_counter)
        
        # Find the best solution
        for solution in population:
            fitness = calculate_fitness(solution)
            if fitness > best_fitness:
                best_fitness = fitness
                best_solution = solution
    
    return best_solution, best_fitness

best_solution, best_fitness = abc_algorithm()

print(f"Best Fitness: {best_fitness}")

# Summarize the solution
assignment = np.argmax(best_solution, axis=0)
for t in range(T):
    print(f"Task {t} is assigned to Worker {assignment[t]} with efficiency {efficiency_matrix[assignment[t], t]}")


Best Fitness: 82.20634089323454
Task 0 is assigned to Worker 3 with efficiency 9.52591939040752
Task 1 is assigned to Worker 3 with efficiency 7.931212992189091
Task 2 is assigned to Worker 2 with efficiency 6.766931414984551
Task 3 is assigned to Worker 1 with efficiency 9.610360770259474
Task 4 is assigned to Worker 1 with efficiency 9.515518907717567
Task 5 is assigned to Worker 3 with efficiency 7.94446892069016
Task 6 is assigned to Worker 0 with efficiency 7.122540811613591
Task 7 is assigned to Worker 4 with efficiency 9.739347595625453
Task 8 is assigned to Worker 1 with efficiency 6.455778763734856
Task 9 is assigned to Worker 2 with efficiency 7.5942613260122735
